In [ ]:
!pip install gensim

In [ ]:
!pip install nltk

In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import gensim
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import GradientBoostingClassifier

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Embedding, Conv1D, LSTM, Bidirectional, GlobalMaxPooling1D, Dense, Dropout, Conv2D, concatenate, MaxPooling2D, Input, Flatten, Reshape
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam



In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
bully_df = pd.read_csv('/content/drive/MyDrive/datasets/cyberbullying_tweets.csv')

In [ ]:
def cleantxt(txt):
    stpw = stopwords.words('english')

    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-zA-Z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt

In [ ]:
bully_df.tail()

In [ ]:
bully_df = bully_df.rename(columns={'tweet_text':'tweet','cyberbullying_type':'type'})

In [ ]:
bully_df.head()

In [ ]:
bully_df['tweet'] = bully_df['tweet'].apply(cleantxt)

In [ ]:
bully_df.head()

In [ ]:
bully_df.tail(20)

In [ ]:
import sys
import nltk

print("Python Version:", sys.version)
print("NLTK Version:", nltk.__version__)

In [ ]:
import nltk
import zipfile
import os
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

print(nltk.data.path)

# nltk_data_path = "/root/nltk_data/corpora/corpora/corpora"
# os.makedirs(nltk_data_path, exist_ok=True)

# # Set the environment variable
# os.environ['NLTK_DATA'] = nltk_data_path
# nltk.data.path.append(nltk_data_path)


# nltk.download('wordnet', download_dir=nltk_data_path)
# nltk.download('stopwords', download_dir=nltk_data_path)
# nltk.download('omw-1.4', download_dir=nltk_data_path)
# nltk.download('punkt', download_dir=nltk_data_path)
# nltk.download('punkt_tab')

# wordnet_zip_path = os.path.join(nltk_data_path, "wordnet.zip")
# extract_path = os.path.join(nltk_data_path)

# # Extract the ZIP file
# with zipfile.ZipFile(wordnet_zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# try:
#     print(wordnet.synsets("computer"))  # Test if WordNet works
# except LookupError:
#     print("WordNet not found!")



def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(sentence)
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in tokens]
    return ' '.join(lemmatized_words)

bully_df['tweet'] = bully_df['tweet'].apply(lemmatize_sentence)

In [ ]:
bully_df.tail(10)

In [ ]:
bully_df.tail(10)

In [ ]:
# from wordcloud import WordCloud
# negative_words = ' '.join([text for text in bully_df['tweet'][bully_df['type'] != 'not_cyberbullying']])
# wordcloud = WordCloud(width=800, height=500,
# random_state=21, max_font_size=110).generate(negative_words)
# plt.figure(figsize=(10, 7))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis('off')
# plt.show()

In [ ]:
bad_word = ['fuck','dumb','black','nigger','terrorist','bully','stupid','rape','joke','racism','islam','idiot','isis','bitch','hate','muslim','racist','hindu','christian','kill','sexist','shit','wrong','terrorism']

In [ ]:
bully_df['tweetsplit'] = bully_df['tweet'].apply(lambda x: x.split())
# bully_df['tweet'] = bully_df['tweet'].apply(lambda x: ([w for w in x if len(w)>=3]))
# bully_df['tweetcnn'] = bully_df['tweet'].apply(lambda x: ' '.)

In [ ]:
bully_df.tail(20)

In [ ]:
# gender religion age ethnicity other_cyberbullying
not_bully_df = bully_df[bully_df['type'] == 'not_cyberbullying']
gender_bully_df = bully_df[bully_df['type'] == 'gender']
religion_bully_df = bully_df[bully_df['type'] == 'religion']
age_bully_df = bully_df[bully_df['type'] == 'age']
ethnicity_bully_df = bully_df[bully_df['type'] == 'ethnicity']
other_bully_df = bully_df[bully_df['type'] == 'other_cyberbullying']
all_bully_df = bully_df[bully_df['type'] != 'not_cyberbullying']

print(not_bully_df.shape)
print(gender_bully_df.shape)
print(religion_bully_df.shape)
print(age_bully_df.shape)
print(ethnicity_bully_df.shape)
print(other_bully_df.shape)
print(all_bully_df.shape)

In [ ]:
not_bully_df.tail()

In [ ]:
# sampled_not_bully_df = not_bully_df.sample(n=2000, random_state=42)
# sampled_gender_bully_df = gender_bully_df.sample(n=1200, random_state=42)
# sampled_age_bully_df = age_bully_df.sample(n=1200, random_state=42)
# sampled_religion_bully_df = religion_bully_df.sample(n=1200, random_state=42)
# sampled_ethnicity_bully_df = ethnicity_bully_df.sample(n=1200, random_state=42)
# sampled_other_bully_df = other_bully_df.sample(n=1200, random_state=42)

sampled_not_bully_df = not_bully_df.sample(n=7000, random_state=42)
sampled_gender_bully_df = gender_bully_df.sample(n=1750, random_state=42)
sampled_age_bully_df = age_bully_df.sample(n=1750, random_state=42)
sampled_religion_bully_df = religion_bully_df.sample(n=1750, random_state=42)
sampled_ethnicity_bully_df = ethnicity_bully_df.sample(n=1750, random_state=42)
sampled_other_bully_df = []

sampled2_not_bully_df = not_bully_df.sample(n=3000, random_state=42)
sampled2_gender_bully_df = gender_bully_df.sample(n=600, random_state=42)
sampled2_age_bully_df = age_bully_df.sample(n=600, random_state=42)
sampled2_religion_bully_df = religion_bully_df.sample(n=600, random_state=42)
sampled2_ethnicity_bully_df = ethnicity_bully_df.sample(n=600, random_state=42)
sampled2_other_bully_df = other_bully_df.sample(n=600, random_state=42)

In [ ]:
sampled_gender_bully_df.shape

In [ ]:
sampled_gender_bully_df.tail()

In [ ]:
final_df = pd.concat([sampled_not_bully_df, sampled_gender_bully_df, sampled_age_bully_df, sampled_religion_bully_df, sampled_ethnicity_bully_df], ignore_index=True)
# final_df['tweet'] = final_df['tweetsplit'].apply(lambda x: ([w for w in x if len(w)>=3]))

In [ ]:
final2_df = pd.concat([sampled2_not_bully_df, sampled2_gender_bully_df, sampled2_age_bully_df, sampled2_religion_bully_df, sampled2_ethnicity_bully_df, sampled2_other_bully_df], ignore_index=True)
# final2_df['tweet'] = final2_df['tweetsplit'].apply(lambda x: ([w for w in x if len(w)>=3]))

In [ ]:
final_df.tail(20)

In [ ]:
maximum_length=356
# xx = pad_sequences(final_df['tweetcnn'], maxlen=maximum_length)


In [ ]:
final2_df.loc[final2_df['type']!='not_cyberbullying','type'] = 'yes'
final2_df.loc[final2_df['type']=='not_cyberbullying','type'] = 'no'

In [ ]:
# def allign(text):
#   while len(text)<5:
#     text.append('<oov>')
#   return text

# final_df['tweet'] = final_df['tweet'].apply(allign)
# final2_df['tweet'] = final2_df['tweet'].apply(allign)

In [ ]:
mapping = {'not_cyberbullying': 0, 'gender': 1, 'age': 1, 'ethnicity': 1, 'religion':1, 'other_cyberbullying':1}
mapping2 = {'yes': 1, 'no': 0}

In [ ]:
final_df.tail(20)

In [ ]:
final_df['type'] = final_df['type'].map(mapping)
final2_df['type'] = final2_df['type'].map(mapping2)

In [ ]:
final_df.tail(20)

In [ ]:
final_df.shape

In [ ]:
def tokenize_texts(sentences):
  NUM_WORDS = 30000

  # Define/Load Tokenize text function
  tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                        lower=True)

  # Fit the function on the text
  tokenizer.fit_on_texts(sentences)

  # Count number of unique tokens
  word_index = tokenizer.word_index
  print('Found %s unique tokens.'  %len(word_index))
  return tokenizer, word_index

In [ ]:
tokenizer, word_index = tokenize_texts(final_df['tweet'])
final_df['tweetseq'] = tokenizer.texts_to_sequences(final_df['tweet'])

In [ ]:
print(final_df['tweetseq'].dtype)

In [ ]:
final_df.tail(20)

In [ ]:
print(final_df['tweetseq'].shape)

In [ ]:
# def pad(seq):
#   print(seq)
#   return pad_sequences(seq,maxlen=150)

def padding(sentences):
  seq=sentences.tolist()
  padded = pad_sequences(seq,maxlen=250,padding='post')
  return padded

padded = padding(final_df['tweetseq'])
print(f"Min value: {padded.min()}, Max value: {padded.max()}")

In [ ]:
padded.shape

In [ ]:
final_df['tweetseq'] = list(padded)

In [ ]:
final_df.tail(20)

In [ ]:
word_vectors = dict()

# load the whole embedding into memory
f = open('/content/drive/MyDrive/datasets/glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_vectors[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(word_vectors))

In [ ]:
NUM_WORDS=len(word_index)+1
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,(NUM_WORDS))

embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

In [ ]:
wp=[]
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    if word in word_vectors:
      embedding_vector = word_vectors[word]
      embedding_matrix[i] = embedding_vector
    else:
      fl=0
      for x in bad_word:
        if x in word:
          embedding_vector = word_vectors[x]
          embedding_matrix[i] = embedding_vector
          fl=1
          break
      if fl==0 :
        wp.append(word)
    # try:
    #     embedding_vector = word_vectors[word]
    #     embedding_matrix[i] = embedding_vector
    # except KeyError:
    #     wp.append(word)
    #     embedding_matrix[i]=np.zeros(300)
# ' '.join(wp)

In [ ]:
final_df['type'].value_counts()

In [ ]:
sequence_length=150
drop=0.2
model = Sequential()
NUM_WORDS=len(word_index)+1

model.add(Embedding(input_dim=NUM_WORDS,
                    output_dim=EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=sequence_length,
                    trainable=True))

model.add(Bidirectional(LSTM(512, return_sequences = True)))
model.add(GlobalMaxPooling1D())



model.add(Dense(40, activation='relu'))

model.add(Dense(20, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
X = np.array(final_df['tweetseq'].tolist(), dtype=np.float64)
y=final_df['type']
yc=y

In [ ]:
print(X.shape,yc.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, yc, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [ ]:
from sklearn.utils import class_weight
yy=np.unique(y_train)
opt = Adam(learning_rate=1e-3)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',        # Filepath to save the model
    monitor='val_loss',             # Metric to monitor (e.g., 'val_accuracy', 'val_loss')
    save_best_only=True,            # Save only the best model
    mode='min',                     # 'min' for loss, 'max' for accuracy
    verbose=1                       # Display messages
)

callbacks = [EarlyStopping(monitor='val_loss'),checkpoint]
hist_adam = model.fit(X_train, y_train, batch_size=500, epochs=30, validation_data=(X_val, y_val), callbacks=callbacks)  # starts training

In [ ]:
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist_adam.history['loss'], color='b', label='Training Loss')
plt.plot(hist_adam.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

In [ ]:
best_model = load_model('best_model.keras')
prob = best_model.predict(X_test)

y_pred=[]
for xx in prob:
  if xx>=0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

# y_pred
yy=y_test


cnf_matrix = metrics.confusion_matrix(yy, y_pred)
cnf_matrix



In [ ]:
class_names=['no_bully', 'bully'] # name  of classes
fig, ax = plt.subplots(figsize=(3,3))

# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu", fmt='g', cbar=False)
ax.set_xticklabels(class_names, rotation=45, ha='right')  # Rotate x-ticks for readability
ax.set_yticklabels(class_names, rotation=0)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label(cnn)')


In [ ]:
import pickle

pickl = {
    'tokenizer': tokenizer,
    'model': best_model
}

pickle.dump(pickl, open("lstm"+".p","wb"))

target_names = ['no_bully', 'bully']
print(classification_report(yy, y_pred, target_names=target_names))

In [ ]:
accuracy = accuracy_score(yy, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

In [ ]:
TP = cnf_matrix[0, 0]
FN = cnf_matrix[0, 1]
FP = cnf_matrix[1, 0]
TN = cnf_matrix[1, 1]

recall = TP / (TP + FN)

precision = TP / (TP + FP)

f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

In [ ]:

# Plot ROC curve for each class
plt.figure(figsize=(6,5))
yy=[]
yy.append(y_test)
yy.append(1-y_test)
pr=[]
pr.append(prob)
pr.append(1-prob)

for i in range(2):
    fpr, tpr, _ = roc_curve(yy[i], pr[i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

# Plot the diagonal line for random guessing
plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')

# Label the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Each Class')
plt.legend(loc='best')
plt.show()